In [1]:
import hail as hl
import gnomad

In [2]:
def create_grch38_revel_info(path, ow = False, give_object = False):
    revel_information = hl.import_table("gs://gnomad-wphu/revel_grch38_all_chromosomes.csv", delimiter=",", types={'hg19_pos':hl.tint,'grch38_pos':hl.tstr,'REVEL': hl.tfloat64})
    revel_information_grch38 = revel_information.drop("hg19_pos")
    revel_information_grch38 = revel_information_grch38.filter(revel_information_grch38.grch38_pos.contains("."),keep=False)
    revel_information_grch38 = revel_information_grch38.annotate(grch38_pos_int = hl.int(revel_information_grch38.grch38_pos))
    revel_information_grch38 = revel_information_grch38.transmute(grch38_pos = revel_information_grch38.grch38_pos_int)
    revel_information_grch38 = revel_information_grch38.transmute(chr = "chr" + revel_information_grch38.chr)
    revel_information_grch38 = revel_information_grch38.annotate(locus = hl.locus(revel_information_grch38.chr, revel_information_grch38.grch38_pos, reference_genome="GRCh38"))
    revel_information_grch38 = revel_information_grch38.annotate(alleles = hl.array([revel_information_grch38.ref, revel_information_grch38.alt]))
    revel_information_grch38 = revel_information_grch38.select("locus","alleles","REVEL", "aaref", "aaalt")
    revel_information_grch38 = revel_information_grch38.key_by("locus", "alleles")
    if ow:
        revel_information_grch38.write(path, overwrite=ow)
    if give_object:
        return revel_information_grch38
    
def load_grch38_revel_info():
    return(hl.read_table("gs://gnomad-wphu/revel_annotations_grch38.ht"))

In [3]:
from gnomad.utils.vcf import ht_to_vcf_mt
def get_chr():
    info_ht = gnomad_vars.select()
    info_ht = info_ht.filter(info_ht.locus.contig=="chr1")
    info_ht = info_ht.head(100000000)
    hl.export_vcf(info_ht, "gs://gnomad-wphu/info_split_large_chr1.vcf")

In [4]:
def create_cadd_info():
    cadd = hl.import_table("gs://gnomad-wphu/gnomad.genomes.r3.0.indel.tsv", comment = "#", types={'Pos':hl.tint32, "RawScore":hl.tfloat, "PHRED":hl.tfloat})
    cadd = cadd.transmute(Chrom = "chr" + cadd.Chrom)
    cadd = cadd.annotate(locus = hl.locus(cadd.Chrom, cadd.Pos, reference_genome="GRCh38"))
    cadd = cadd.annotate(alleles = hl.array([cadd.Ref, cadd.Alt]))
    cadd = cadd.select("locus","alleles","RawScore", "PHRED")
    cadd = cadd.key_by("locus", "alleles")
    return cadd


In [5]:
def load_CADD(path, n_partitions, force_bgz = False):
    column_names = {'f0': 'chrom', 'f1': 'pos', 'f2': 'ref', 'f3': 'alt', 'f4': 'RawScore', 'f5': 'PHRED'}
    types = {'f0': hl.tstr, 'f1': hl.tint32, 'f4': hl.tfloat32, 'f5': hl.tfloat32}
    cadd_ht = hl.import_table(path, comment="#", no_header=True, types=types, min_partitions=n_partitions, force_bgz = force_bgz)
    cadd_ht = cadd_ht.rename(column_names)
    chrom = hl.format("chr%s", cadd_ht.chrom)
    locus = hl.locus(chrom, cadd_ht.pos, reference_genome="GRCh38")
    alleles = hl.array([cadd_ht.ref, cadd_ht.alt])
    cadd_ht = cadd_ht.transmute(locus=locus, alleles=alleles)
    cadd_union_ht = cadd_ht.head(0)
    for contigs in (range(1, 10), list(range(10, 23)) + ["X", "Y", "MT"]):
        contigs = ["chr%s" % contig for contig in contigs]
        cadd_ht_subset = cadd_ht.filter(hl.array(list(map(str, contigs))).contains(cadd_ht.locus.contig))
        cadd_union_ht = cadd_union_ht.union(cadd_ht_subset)

    cadd_union_ht = cadd_union_ht.select("locus", "alleles", "RawScore", "PHRED")
    cadd_union_ht = cadd_union_ht.key_by("locus", "alleles")
    cadd_union_ht = cadd_union_ht.annotate_globals(source_file_path = path)

    cadd_union_ht.describe()

    return cadd_union_ht

def make_unified_CADD():
    snvs = hl.read_table("gs://gnomad-wphu/CADD-v1.6-SNVs.ht")
    release3_indels = hl.read_table("gs://gnomad-wphu/CADD-v1.6-indels-updated.ht")
    raw31_indels = hl.read_table("gs://gnomad-wphu/CADD-indels-gnomad.3.1.ht")
    raw31_complex = hl.read_table("gs://gnomad-wphu/CADD-1.6-gnomad-complex-variants.ht")
    unified = snvs.head(0)
    unified = unified.union(snvs,release3_indels,raw31_indels, raw31_complex)
    unified = unified.annotate_globals(source_file_path = {"snvs":"gs://gnomad-wphu/CADD-v1.6-SNVs.ht",
                                                         "v3-indels":"gs://gnomad-wphu/CADD-v1.6-indels-updated.ht",
                                                         "v3.1-indels":"gs://gnomad-wphu/CADD-indels-gnomad.3.1.ht",
                                                         "v3.1-complex":"gs://gnomad-wphu/CADD-1.6-gnomad-complex-variants.ht"
                                                        })
    #unified.describe()
    return unified

def convert_CADD_indels_64_32():
    release3_indels = hl.read_table("gs://gnomad-wphu/CADD-v1.6-indels.ht")
    release3_indels = release3_indels.transmute(RawScore = hl.float32(release3_indels.RawScore))
    release3_indels = release3_indels.transmute(PHRED = hl.float32(release3_indels.PHRED))
    release3_indels.describe()
    release3_indels.write("gs://gnomad-wphu/CADD-v1.6-indels-updated.ht", overwrite = True)
    
#unified_CADD = make_unified_CADD()
#unified_CADD = unified_CADD.write("gs://gnomad-wphu/complete-CADD-v1.6-annotations.ht", overwrite=True)

In [6]:
def export_for_CADD_analysis(hl_tbl,path):
    export = hl_tbl.select()
    export = export.filter(export.locus.contig=="chrM",keep=False)
    hl.methods.export_vcf(export, path)

In [7]:
def combine_splice_ai():
    recode = {f"{i}":f"chr{i}" for i in (list(range(1, 23)) + ['X', 'Y'])}
    splice_snps_skip_invalid = hl.import_vcf("gs://gnomad-wphu/splice_ai_data/splice_ai_data/genome_scores_v1.3_ds.20a701bc58ab45b59de2576db79ac8d0/spliceai_scores.masked.snv.hg38.vcf.gz",
                                force_bgz= True,
                                min_partitions=3000,
                                reference_genome='GRCh38', contig_recoding=recode, skip_invalid_loci= True
                               )
    splice_snps_skip_invalid.annotate_globals(source_file_path="gs://gnomad-wphu/splice_ai_data/splice_ai_data/genome_scores_v1.3_ds.20a701bc58ab45b59de2576db79ac8d0/spliceai_scores.masked.snv.hg38.vcf.gz")
    
    splice_indels_skip_invalid = hl.import_vcf("gs://gnomad-wphu/splice_ai_data/gnomAD_v3.1_SpliceAI_scores-selected/spliceai_scores.masked.gnomad_indel.hg38.vcf.gz",
                                   force_bgz=True,
                                   reference_genome='GRCh38', contig_recoding=recode,skip_invalid_loci=True,
                                   min_partitions=1000)
    splice_indels_skip_invalid.annotate_globals(source_file_path = "gs://gnomad-wphu/splice_ai_data/gnomAD_v3.1_SpliceAI_scores-selected/spliceai_scores.masked.gnomad_indel.hg38.vcf.gz")
    
    spliceAi_info_skip_invalid = splice_snps_skip_invalid.union_rows(splice_indels_skip_invalid)
    return spliceAi_info_skip_invalid

def annotate_spliceAi(mt):
    delta_scores = mt.info.SpliceAI[0].split(delim="\\|")[2:6]
    splice_split = mt.info.annotate(
        SpliceAI=hl.map(lambda x: hl.float32(x), delta_scores)
    ).rename({"SpliceAI":"splice_ai"})
    mt = mt.annotate_rows(info=splice_split)

    # Annotate info.max_DS with the max of DS_AG, DS_AL, DS_DG, DS_DL in info.
    # delta_score array is |DS_AG|DS_AL|DS_DG|DS_DL
    consequences = hl.literal(
        ["acceptor_gain", "acceptor_loss", "donor_gain", "donor_loss"]
    )
    mt = mt.annotate_rows(info=mt.info.annotate(max_ds=hl.max(mt.info.splice_ai)))
    mt = mt.annotate_rows(
        info=mt.info.annotate(
            splice_consequence=hl.if_else(
                mt.info.max_ds > 0,
                consequences[mt.info.splice_ai.index(mt.info.max_ds)],
                "no_consequence",
            )
        )
    )

    return mt
#spliceAi_info_skip_invalid = combine_splice_ai()
#spliceAi_info_skip_invalid = annotate_spliceAi(spliceAi_info_skip_invalid)
#spliceAi_info_skip_invalid.write("gs://gnomad-wphu/spliceai-scores-updated.ht", overwrite=True)


In [8]:
#chr, pos, ref, alt, refAA, altAA, strand_1pos_0neg, trinucleotide_context, UCSC_gene, ExAC_coverage,primateDL_score
def create_primate_ai_info(write = False, path = "", rewrite = False):
    primate_ai= hl.import_table("gs://gnomad-wphu/PrimateAI_scores_v0.2_hg38.tsv.gz",
                                          force=True, comment="#", skip_blank_lines=True,
                                          types={"pos":hl.tint32, 'primateDL_score':hl.tfloat32, 'ExAC_coverage':hl.tfloat32}
                                         )
    primate_ai = primate_ai.annotate_globals(source_file_path="gs://gnomad-wphu/PrimateAI_scores_v0.2_hg38.tsv.gz")
    primate_ai = primate_ai.annotate(locus = hl.locus(primate_ai.chr,primate_ai.pos, reference_genome="GRCh38"))
    primate_ai = primate_ai.annotate(alleles = hl.array([primate_ai.ref, primate_ai.alt]))
    primate_ai = primate_ai.select("locus", "alleles", "primateDL_score", "ExAC_coverage","strand_1pos_0neg","refAA", "altAA", "trinucleotide_context", "UCSC_gene")
    primate_ai = primate_ai.key_by("locus","alleles")
    if primate_ai.n_partitions() < 10:
        primate_ai = primate_ai.repartition(500)
    if write:
        if overwrite:
            primate_ai.write(path, overwrite = rewrite)
        else:
            try:
                primate_ai.write(path)
            except Exception as e: print(e)
    primate_ai.describe()
    return primate_ai


In [9]:
gnomad_variants = hl.read_table("gs://gnomad/annotations/hail-0.2/ht/genomes_v3.1/gnomad_genomes_v3.1_info.split.ht")
gnomad_indels = hl.read_table("gs://gnomad-wphu/gnomad_indels.ht")
gnomad_variants_v3 = hl.read_table("gs://gnomad/annotations/hail-0.2/ht/genomes_v3/gnomad_genomes_v3_info.split.ht")
gnomad3_release = hl.read_table("gs://gnomad-public-requester-pays/release/3.0/ht/genomes/gnomad.genomes.r3.0.sites.ht")

#seqr_annotations_tbl = hl.read_table("gs://seqr-reference-data/GRCh38/all_reference_data/v2/combined_reference_data_grch38-2.0.3.ht")
#gnomad_vars_in_seqr = gnomad_variants.semi_join(seqr_annotations_tbl)
#gnomad_vars_not_in_seqr = gnomad_variants.anti_join(seqr_annotations_tbl)

#cadd_tbl = hl.read_table("gs://seqr-reference-data/GRCh38/CADD/CADD_snvs_and_indels.v1.4.ht")
#CADD_indels = create_cadd_indel_info()
CADD_indels = hl.read_table("gs://gnomad-wphu/CADD-v1.6-indels-updated.ht")
gnomad_indels_not_in_CADD = hl.read_table("gs://gnomad-wphu/gnomad-indels-anti-CADD.ht")
gnomad_indels_in_CADD = hl.read_table("gs://gnomad-wphu/gnomad-indels-semi-CADD.ht")
CADD_snps = hl.read_table("gs://gnomad-wphu/CADD-v1.6-SNVs.ht")
cadd = hl.read_table("gs://gnomad-wphu/complete-CADD-v1.6-annotations.ht")

primate_ai_info_old = hl.read_table("gs://seqr-reference-data/GRCh38/primate_ai/PrimateAI_scores_v0.2.liftover_grch38.ht")
primate_ai_info = hl.read_table("gs://gnomad-wphu/primate-ai-info.ht")
gnomad_vars_not_in_primate_ai = hl.read_table("gs://gnomad-wphu/gnomad-vars-anti-primate.ht")


#revel_information = hl.import_table("gs://gnomad-wphu/revel_grch38_all_chromosomes.csv", delimiter=",", types={'hg19_pos':hl.tint,'grch38_pos':hl.tstr,'REVEL': hl.tfloat64})
revel_information_grch38 = hl.read_table("gs://gnomad-wphu/revel_annotations_grch38.ht")
gnomad_vars_not_in_revel = hl.read_table("gs://gnomad-wphu/gnomad-vars-anti-revel.ht")
gnomad_vars_in_revel = hl.read_table("gs://gnomad-wphu/gnomad-vars-semi-revel.ht")

spliceAi_info_skip_invalid = hl.read_matrix_table("gs://gnomad-wphu/spliceai-scores-updated.ht")
spliceAi_snps_skip_invalid = hl.read_matrix_table("gs://gnomad-wphu/splice_snps_skip_invalid.ht")
spliceAi_indels_skip_invalid = hl.read_matrix_table("gs://gnomad-wphu/splice_indels_skip_invalid.ht")

full_annotations = hl.read_table("gs://gnomad/annotations/hail-0.2/ht/genomes_v3.1/gnomad_genomes_v3.1.analyst_annotations.ht")


Initializing Hail with default parameters...
Running on Apache Spark version 2.4.5
SparkUI available at http://wphu-m.c.broad-mpg-gnomad.internal:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.57-582b2e31b8bd
LOGGING: writing to /home/hail/hail-20201207-1818-0.2.57-582b2e31b8bd.log


In [50]:
#gnomad_vars_in_revel = gnomad_variants.semi_join(revel_information_grch38)
#gnomad_vars_not_in_revel = gnomad_variants.anti_join(revel_information_grch38)

#gnomad_vars_not_in_revel = gnomad_vars_not_in_revel.checkpoint("gs://gnomad-wphu/gnomad-vars-anti-revel.ht")
#gnomad_indels_in_revel = gnomad_vars_in_revel.checkpoint("gs://gnomad-wphu/gnomad-vars-semi-revel.ht")

#gnomad_indels = gnomad_indels.checkpoint("gs://gnomad-wphu/gnomad-indels-v3.1.ht")
#gnomad_indels_not_in_CADD = gnomad_indels.anti_join(CADD_indels)
#gnomad_indels_in_CADD = gnomad_indels.semi_join(CADD_indels)

#gnomad_indels_not_in_CADD = gnomad_indels_not_in_CADD.checkpoint("gs://gnomad-wphu/gnomad-indels-anti-CADD.ht")
#gnomad_indels_in_CADD = gnomad_indels_in_CADD.checkpoint("gs://gnomad-wphu/gnomad-indels-semi-CADD.ht")


#gnomad_indels = gnomad_indels.checkpoint("gs://gnomad-wphu/gnomad_indels.ht", overwrite = True)
#gnomad_indels.count()
#gnomad_indels.write("gs://gnomad-wphu/gnomad_indels.ht", overwrite = True)

#gnomad3_release = hl.read_table("gs://gnomad-public-requester-pays/release/3.0/ht/genomes/gnomad.genomes.r3.0.sites.ht")
#gnomad3_release.describe()

#gnomad3_release_indels = gnomad3_release.filter(
                            #hl.is_indel(gnomad3_release.alleles[0], gnomad3_release.alleles[1]))

#gnomad3_release_indels.count()

In [17]:
def annotate_gnomad_v31(list_of_annotations, gnomad_v31_tbl):
    import re
    global_annotations = {}
    for annotation in list_of_annotations:
        annotation = re.sub(r'(_| |-)',"",annotation.lower())
        print(annotation)
        if annotation == "cadd":
            cadd = hl.read_table("gs://gnomad-wphu/complete-CADD-v1.6-annotations.ht")
            cadd = cadd.transmute(cadd = hl.struct(raw_score = cadd.RawScore,
                                                   phred = cadd.PHRED
                                                  ))
            #CADD = hl.read_table()
            #gnomad_v31_tbl = gnomad_v31_tbl.join(CADD.transmute(CADD = hl.struct(RawScore = CADD.RawScore, PHRED = CADD.PHRED)), how="left")
            gnomad_v31_tbl = gnomad_v31_tbl.join(cadd, how = "left")
            global_annotations["cadd"]="gs://gnomad-wphu/complete-CADD-v1.6-annotations.ht"
            
        elif annotation == "revel":
            revel = hl.read_table("gs://gnomad-wphu/revel_annotations_grch38.ht")
            revel = revel.transmute(revel = hl.struct(revel_score = revel.REVEL,
                                                      ref_aa = revel.aaref,
                                                      alt_aa = revel.aaalt
                                                     ))
            gnomad_v31_tbl = gnomad_v31_tbl.join(revel, how = "left")
            global_annotations["revel"] = "gs://gnomad-wphu/revel_annotations_grch38.ht"
            
        elif annotation == "spliceai":
            spliceai = hl.read_matrix_table("gs://gnomad-wphu/spliceai-scores-updated.ht")
            #spliceai = spliceai.annotate_rows(info = spliceai.info.annotate(
                                                #risd = spliceai.rsid,
                                                #qual = spliceai.qual,
                                                #filters = spliceai.filters
                                                #)
                                             #)
            spliceai = spliceai.rename({"info" : "splice_ai"})
            gnomad_v31_tbl = gnomad_v31_tbl.join(spliceai.make_table(), how = "left")
            global_annotations["splice_ai"] = "gs://gnomad-wphu/spliceai-scores-updated.ht"
            
        elif annotation == "primateai":
            primateai = hl.read_table("gs://gnomad-wphu/primate-ai-info.ht")
            primateai = primateai.transmute(primate_ai = hl.struct(primate_ai_score = primateai.primateDL_score))
            gnomad_v31_tbl = gnomad_v31_tbl.join(primateai, how = "left")
            global_annotations["primate_ai"] = "gs://gnomad-wphu/primate-ai-info.ht"
    gnomad_v31_tbl = gnomad_v31_tbl.select_globals()
    gnomad_v31_tbl = gnomad_v31_tbl.annotate_globals(annotation_file_path=global_annotations)
    gnomad_v31_tbl.describe()
    gnomad_v31_tbl = gnomad_v31_tbl.select("cadd", "revel", "splice_ai", "primate_ai")
    return gnomad_v31_tbl
        
result = annotate_gnomad_v31(["CADD", "REVEL", "SPLICE-AI", "PRIMATE-AI"], gnomad_variants)
result.describe()


cadd


2020-12-07 18:29:04 Hail: INFO: Table.join: renamed the following fields on the right to avoid name conflicts:
    'alleles' -> 'alleles_1'
    'locus' -> 'locus_1'


1570602305
revel


2020-12-07 18:38:54 Hail: INFO: Table.join: renamed the following fields on the right to avoid name conflicts:
    'alleles' -> 'alleles_1'
    'locus' -> 'locus_1'


1571019842
spliceai


2020-12-07 18:52:42 Hail: INFO: Table.join: renamed the following fields on the right to avoid name conflicts:
    'locus' -> 'locus_1'
    'alleles' -> 'alleles_1'


1584070311
primateai


2020-12-07 19:11:31 Hail: INFO: Table.join: renamed the following fields on the right to avoid name conflicts:
    'alleles' -> 'alleles_1'
    'locus' -> 'locus_1'


1584091437
----------------------------------------
Global fields:
    'annotation_file_path': dict<str, str> 
----------------------------------------
Row fields:
    'locus': locus<GRCh38> 
    'alleles': array<str> 
    'info': struct {
        ReadPosRankSum: float64, 
        MQRankSum: float64, 
        QUALapprox: int64, 
        VarDP: int32, 
        SB: array<int32>, 
        MQ: float64, 
        QD: float32, 
        FS: float64, 
        AS_ReadPosRankSum: float64, 
        AS_MQRankSum: float64, 
        AS_QUALapprox: int64, 
        AS_VarDP: int32, 
        AS_MQ: float64, 
        AS_QD: float32, 
        AS_SB_TABLE: array<int32>, 
        AS_FS: float64, 
        AC_raw: int32, 
        AC: int32, 
        AS_pab_max: float64
    } 
    'lowqual': bool 
    'AS_lowqual': bool 
    'a_index': int32 
    'was_split': bool 
    'old_locus': locus<GRCh38> 
    'old_alleles': array<str> 
    'cadd': struct {
        raw_score: float32, 
        phred: float32
    } 
    

In [18]:
#r = gnomad_variants.anti_join(full_annotations)
#r.show()

+---------------+------------+---------------------+----------------+-----------------+------------+--------------+---------+---------+
| locus         | alleles    | info.ReadPosRankSum | info.MQRankSum | info.QUALapprox | info.VarDP | info.SB      | info.MQ | info.QD |
+---------------+------------+---------------------+----------------+-----------------+------------+--------------+---------+---------+
| locus<GRCh38> | array<str> |             float64 |        float64 |           int64 |      int32 | array<int32> | float64 | float32 |
+---------------+------------+---------------------+----------------+-----------------+------------+--------------+---------+---------+
+---------------+------------+---------------------+----------------+-----------------+------------+--------------+---------+---------+

+---------+------------------------+-------------------+--------------------+---------------+------------+------------+------------------+
| info.FS | info.AS_ReadPosRankSum | info.AS_MQRankSum | info.AS_QUALapprox | info.AS_VarDP | info.AS_MQ | info.AS_QD | info.AS_SB_TABLE |
+---------+------------------------+-------------------+--------------------+---------------+------------+------------+------------------+
| float64 |                float64 |           float64 |              int64 |         int32 |    float64 |    float32 | array<int32>     |
+---------+------------------------+-------------------+--------------------+---------------+------------+------------+------------------+
+---------+------------------------+-------------------+--------------------+---------------+------------+------------+------------------+

+------------+-------------+---------+-----------------+---------+------------+---------+-----------+---------------+-------------+
| info.AS_FS | info.AC_raw | info.AC | info.AS_pab_max | lowqual | AS_lowqual | a_index | was_split | old_locus     | old_alleles |
+------------+-------------+---------+-----------------+---------+------------+---------+-----------+---------------+-------------+
|    float64 |       int32 |   int32 |         float64 |    bool |       bool |   int32 |      bool | locus<GRCh38> | array<str>  |
+------------+-------------+---------+-----------------+---------+------------+---------+-----------+---------------+-------------+
+------------+-------------+---------+-----------------+---------+------------+---------+-----------+---------------+-------------+

In [15]:
#full_annotations.count()

1584091437

In [16]:
#gnomad_variants.count()

1570602305

In [46]:
#test = gnomad_variants
#print(test.count())
#revel = hl.read_table("gs://gnomad-wphu/revel_annotations_grch38.ht")
#revel = revel.transmute(revel = hl.struct(revel_score = revel.REVEL,
#                                                      ref_aa = revel.aaref,
#                                                      alt_aa = revel.aaalt
#                                                     ))
#test = test.join(revel, how = "left")
#test.count()

1570602305


2020-12-07 20:23:50 Hail: INFO: Table.join: renamed the following fields on the right to avoid name conflicts:
    'alleles' -> 'alleles_1'
    'locus' -> 'locus_1'


1571019842

In [47]:
#r = test.anti_join(gnomad_variants)
#r.count()

0

In [50]:
#test.distinct().count()

1570602305

In [77]:
#a = full_annotations.collect_by_key()
#b = a.filter(hl.len(a.values)>1)
#full_annotations.semi_join(b).show()

+---------------+------------+----------------+------------+-------------------+--------------+--------------+
| locus         | alleles    | cadd.raw_score | cadd.phred | revel.revel_score | revel.ref_aa | revel.alt_aa |
+---------------+------------+----------------+------------+-------------------+--------------+--------------+
| locus<GRCh38> | array<str> |        float32 |    float32 |           float64 | str          | str          |
+---------------+------------+----------------+------------+-------------------+--------------+--------------+
| chr1:925884   | ["C","T"]  |       5.99e-01 |   7.50e+00 |                NA | NA           | NA           |
| chr1:925884   | ["C","T"]  |       5.99e-01 |   7.50e+00 |                NA | NA           | NA           |
| chr1:925885   | ["C","T"]  |       3.01e-02 |   1.53e+00 |                NA | NA           | NA           |
| chr1:925885   | ["C","T"]  |       3.01e-02 |   1.53e+00 |                NA | NA           | NA           |
| chr1:925886   | ["G","T"]  |       3.78e-01 |   5.18e+00 |                NA | NA           | NA           |
| chr1:925886   | ["G","T"]  |       3.78e-01 |   5.18e+00 |                NA | NA           | NA           |
| chr1:925887   | ["C","T"]  |       8.18e-01 |   9.57e+00 |                NA | NA           | NA           |
| chr1:925887   | ["C","T"]  |       8.18e-01 |   9.57e+00 |                NA | NA           | NA           |
| chr1:925889   | ["C","G"]  |       9.28e-01 |   1.07e+01 |                NA | NA           | NA           |
| chr1:925889   | ["C","G"]  |       9.28e-01 |   1.07e+01 |                NA | NA           | NA           |
+---------------+------------+----------------+------------+-------------------+--------------+--------------+

+---------------------------------------+------------------+------------------------------+-----------------------------+
| splice_ai.splice_ai                   | splice_ai.max_ds | splice_ai.splice_consequence | primate_ai.primate_ai_score |
+---------------------------------------+------------------+------------------------------+-----------------------------+
| array<float32>                        |          float32 | str                          |                     float32 |
+---------------------------------------+------------------+------------------------------+-----------------------------+
| [0.00e+00,0.00e+00,0.00e+00,0.00e+00] |         0.00e+00 | "no_consequence"             |                          NA |
| [4.00e-02,4.00e-02,0.00e+00,0.00e+00] |         4.00e-02 | "acceptor_gain"              |                          NA |
| [0.00e+00,0.00e+00,0.00e+00,0.00e+00] |         0.00e+00 | "no_consequence"             |                          NA |
| [1.00e-02,2.00e-02,0.00e+00,0.00e+00] |         2.00e-02 | "acceptor_loss"              |                          NA |
| [0.00e+00,0.00e+00,0.00e+00,0.00e+00] |         0.00e+00 | "no_consequence"             |                          NA |
| [1.10e-01,5.00e-02,1.00e-02,0.00e+00] |         1.10e-01 | "acceptor_gain"              |                          NA |
| [0.00e+00,0.00e+00,0.00e+00,0.00e+00] |         0.00e+00 | "no_consequence"             |                          NA |
| [3.00e-02,2.00e-02,0.00e+00,0.00e+00] |         3.00e-02 | "acceptor_gain"              |                          NA |
| [0.00e+00,0.00e+00,0.00e+00,0.00e+00] |         0.00e+00 | "no_consequence"             |                          NA |
| [0.00e+00,0.00e+00,0.00e+00,0.00e+00] |         0.00e+00 | "no_consequence"             |                          NA |
+---------------------------------------+------------------+------------------------------+-----------------------------+
showing top 10 rows

In [89]:
#print(spliceAi_info_skip_invalid.make_table().count())
#print(spliceAi_info_skip_invalid.make_table().distinct().count())
#a = spliceAi_info_skip_invalid.make_table().collect_by_key()
#b = a.filter(hl.len(a.values)>1)
#spliceAi_info_skip_invalid.make_table().semi_join(b).show()

3696314616
3602698485


+---------------+------------+------+-----------+----------+---------------------------------------+-------------+-------------------------+
| locus         | alleles    | rsid |      qual | filters  | info.splice_ai                        | info.max_ds | info.splice_consequence |
+---------------+------------+------+-----------+----------+---------------------------------------+-------------+-------------------------+
| locus<GRCh38> | array<str> | str  |   float64 | set<str> | array<float32>                        |     float32 | str                     |
+---------------+------------+------+-----------+----------+---------------------------------------+-------------+-------------------------+
| chr1:925884   | ["C","A"]  | NA   | -1.00e+01 | NA       | [0.00e+00,0.00e+00,0.00e+00,0.00e+00] |    0.00e+00 | "no_consequence"        |
| chr1:925884   | ["C","A"]  | NA   | -1.00e+01 | NA       | [2.00e-02,1.00e-02,0.00e+00,0.00e+00] |    2.00e-02 | "acceptor_gain"         |
| chr1:925884   | ["C","G"]  | NA   | -1.00e+01 | NA       | [0.00e+00,0.00e+00,0.00e+00,0.00e+00] |    0.00e+00 | "no_consequence"        |
| chr1:925884   | ["C","G"]  | NA   | -1.00e+01 | NA       | [0.00e+00,0.00e+00,0.00e+00,0.00e+00] |    0.00e+00 | "no_consequence"        |
| chr1:925884   | ["C","T"]  | NA   | -1.00e+01 | NA       | [0.00e+00,0.00e+00,0.00e+00,0.00e+00] |    0.00e+00 | "no_consequence"        |
| chr1:925884   | ["C","T"]  | NA   | -1.00e+01 | NA       | [4.00e-02,4.00e-02,0.00e+00,0.00e+00] |    4.00e-02 | "acceptor_gain"         |
| chr1:925885   | ["C","A"]  | NA   | -1.00e+01 | NA       | [0.00e+00,0.00e+00,0.00e+00,0.00e+00] |    0.00e+00 | "no_consequence"        |
| chr1:925885   | ["C","A"]  | NA   | -1.00e+01 | NA       | [0.00e+00,0.00e+00,0.00e+00,0.00e+00] |    0.00e+00 | "no_consequence"        |
| chr1:925885   | ["C","G"]  | NA   | -1.00e+01 | NA       | [0.00e+00,0.00e+00,0.00e+00,0.00e+00] |    0.00e+00 | "no_consequence"        |
| chr1:925885   | ["C","G"]  | NA   | -1.00e+01 | NA       | [0.00e+00,2.00e-02,0.00e+00,0.00e+00] |    2.00e-02 | "acceptor_loss"         |
+---------------+------------+------+-----------+----------+---------------------------------------+-------------+-------------------------+
showing top 10 rows

In [113]:
#a = spliceAi_snps_skip_invalid.make_table()
#a.filter(hl.is_defined(a.info.SpliceAI)).semi_join(b).show()

,,,,,
,,,,,info
locus,alleles,rsid,qual,filters,SpliceAI
locus<GRCh38>,array<str>,str,float64,set<str>,array<str>
chr1:925884,"[""C"",""A""]",NA,-1.00e+01,NA,"[""A|AL645608.1|0.00|0.00|0.00|0.00|2|27|12|1""]"
chr1:925884,"[""C"",""A""]",NA,-1.00e+01,NA,"[""A|SAMD11|0.02|0.01|0.00|0.00|14|38|14|38""]"
chr1:925884,"[""C"",""G""]",NA,-1.00e+01,NA,"[""G|AL645608.1|0.00|0.00|0.00|0.00|27|6|12|1""]"
chr1:925884,"[""C"",""G""]",NA,-1.00e+01,NA,"[""G|SAMD11|0.00|0.00|0.00|0.00|38|14|38|14""]"
chr1:925884,"[""C"",""T""]",NA,-1.00e+01,NA,"[""T|AL645608.1|0.00|0.00|0.00|0.00|-2|6|12|1""]"
chr1:925884,"[""C"",""T""]",NA,-1.00e+01,NA,"[""T|SAMD11|0.04|0.04|0.00|0.00|14|38|14|38""]"
chr1:925885,"[""C"",""A""]",NA,-1.00e+01,NA,"[""A|AL645608.1|0.00|0.00|0.00|0.00|1|26|6|11""]"


In [86]:
#print(revel_information_grch38.count())
#print(revel_information_grch38.distinct().count())
#a = revel_information_grch38.collect_by_key()
#b = a.filter(hl.len(a.values)>1)
#revel_information_grch38.semi_join(b).show()

82077491
77843668


,,,,
locus,alleles,REVEL,aaref,aaalt
locus<GRCh38>,array<str>,float64,str,str
chr1:930312,"[""C"",""A""]",9.70e-02,"""P""","""Q"""
chr1:930312,"[""C"",""A""]",3.30e-02,"""R""","""S"""
chr1:930312,"[""C"",""G""]",7.20e-02,"""P""","""R"""
chr1:930312,"[""C"",""G""]",3.80e-02,"""R""","""G"""
chr1:930312,"[""C"",""T""]",7.10e-02,"""P""","""L"""
chr1:930312,"[""C"",""T""]",4.40e-02,"""R""","""C"""
chr1:930315,"[""A"",""C""]",2.60e-01,"""H""","""P"""
chr1:930315,"[""A"",""C""]",4.00e-03,"""T""","""P"""


In [82]:
#print(cadd.count())
#print(cadd.distinct().count())

9079588475
9079588475


In [84]:
#print(primate_ai_info.count())
#print(primate_ai_info.distinct().count())
#a = primate_ai_info.collect_by_key()
#b = a.filter(hl.len(a.values)>1)
#primate_ai_info.semi_join(b).show()

70116384
70073665


,,,,,,,,
locus,alleles,primateDL_score,ExAC_coverage,strand_1pos_0neg,refAA,altAA,trinucleotide_context,UCSC_gene
locus<GRCh38>,array<str>,float32,float32,str,str,str,str,str
chr1:2385807,"[""T"",""C""]",2.33e-01,7.97e+01,"""0""","""Q""","""R""","""CAG""","""uc001ajb.1"""
chr1:2385807,"[""T"",""C""]",2.04e-01,7.97e+01,"""0""","""R""","""G""","""CAG""","""uc010nyy.2"""
chr1:2560633,"[""G"",""C""]",6.57e-01,4.77e+01,"""1""","""V""","""L""","""AGT""","""uc001ajt.1"""
chr1:2560633,"[""G"",""C""]",2.78e-01,4.77e+01,"""1""","""S""","""T""","""AGT""","""uc001ajr.3"""
chr1:12824946,"[""C"",""G""]",3.99e-01,0.00e+00,"""1""","""C""","""W""","""GCT""","""uc001ava.1"""
chr1:12824946,"[""C"",""G""]",4.03e-01,8.88e+01,"""0""","""C""","""S""","""TGT""","""uc001auk.2"""
chr1:12824950,"[""A"",""C""]",3.99e-01,0.00e+00,"""1""","""Y""","""S""","""TAC""","""uc001ava.1"""
chr1:12824950,"[""A"",""C""]",4.13e-01,9.01e+01,"""0""","""C""","""G""","""CTG""","""uc001auk.2"""


# Splice AI

In [65]:
spliceAi_info_skip_invalid.describe()

----------------------------------------
Global fields:
    None
----------------------------------------
Column fields:
    's': str
----------------------------------------
Row fields:
    'locus': locus<GRCh38>
    'alleles': array<str>
    'rsid': str
    'qual': float64
    'filters': set<str>
    'info': struct {
        splice_ai: array<float32>, 
        max_ds: float32, 
        splice_consequence: str
    }
----------------------------------------
Entry fields:
    None
----------------------------------------
Column key: ['s']
Row key: ['locus', 'alleles']
----------------------------------------


In [63]:
spliceAi_info_skip_invalid.describe()

----------------------------------------
Global fields:
    None
----------------------------------------
Column fields:
    's': str
----------------------------------------
Row fields:
    'locus': locus<GRCh38>
    'alleles': array<str>
    'rsid': str
    'qual': float64
    'filters': set<str>
    'info': struct {
        splice_ai: array<float32>, 
        max_DS: float32, 
        splice_consequence: str
    }
----------------------------------------
Entry fields:
    None
----------------------------------------
Column key: ['s']
Row key: ['locus', 'alleles']
----------------------------------------


In [22]:
#splice_indels = hl.import_vcf("gs://gnomad-wphu/splice_ai_data/gnomAD_v3.1_SpliceAI_scores-selected/spliceai_scores.masked.gnomad_indel.hg38.vcf.gz",
                                   #force_bgz=True,
                                   #reference_genome='GRCh38', contig_recoding=recode,skip_invalid_loci=True,
                                   #min_partitions=1000)

#splice_snps = hl.import_vcf("gs://gnomad-wphu/splice_ai_data/splice_ai_data/genome_scores_v1.3_ds.20a701bc58ab45b59de2576db79ac8d0/spliceai_scores.raw.snv.hg38.vcf.gz",
                                #force_bgz= True,
                                #min_partitions=10000,
                                #reference_genome='GRCh38', contig_recoding=recode
                               #)
#splice_snps_withAnno = splice_snps.filter_rows(hl.len(splice_snps.info.SpliceAI)>0, keep=True)

#splice_indels = hl.import_vcf("gs://gnomad-wphu/splice_ai_data/gnomAD_v3.1_SpliceAI_scores-selected/spliceai_scores.masked.gnomad_indel.hg38.vcf.gz",
                                   #force_bgz=True,
                                   #reference_genome='GRCh38', contig_recoding=recode, skip_invalid_loci=True,
                                   #min_partitions=1000)

In [23]:
#print(spliceAi_indels_skip_invalid.count())
#print(spliceAi_snps_skip_invalid.count())
#print(spliceAi_info_skip_invalid.count())

In [24]:
#spliceAi_info_skipInvalid_tbl = spliceAi_info_skipInvalid.make_table()
#spliceAi_info_skipInvalid_tbl.show(5)
#spliceAi_info_skipInvalid_tbl.describe()
#print(gnomad_variants.anti_join(spliceAi_info_skipInvalid_tbl).count())
#gnomad_variants.show(5)
#spliceAi_snps_skipInvalid.show()
#print(gnomad_variants.count())

#numbers do not match because spliceAi only does intergenic variants; should pull up list of genes and compare missing variants against gene list.

In [25]:
#spliceAi_genes = hl.import_table("gs://gnomad-wphu/grch38.tsv", types = {"TX_START":hl.tint64, "TX_END":hl.tint64})
#spliceAi_genes = hl.import_table("gs://gnomad-wphu/grch38.tsv")
#spliceAi_genes = spliceAi_genes.transmute(CHROM = "chr" + spliceAi_genes.CHROM)
#spliceAi_genes = spliceAi_genes.annotate(INTERVAL = hl.parse_locus_interval(spliceAi_genes.CHROM + ":"+spliceAi_genes.TX_START + "-" +spliceAi_genes.TX_END, reference_genome="GRCh38"))

In [26]:
#gnomad_variants_in_genomic_regions = hl.filter_intervals(gnomad_variants, spliceAi_genes.INTERVAL.collect())
#gnomad_variants_in_genomic_regions.count()

In [27]:
#vars_in_genes_not_found = hl.filter_intervals(gnomad_variants.anti_join(spliceAi_info_skipInvalid_tbl),spliceAi_genes.INTERVAL.collect())

In [28]:
#gnomad_indels_not_in_CADD.aggregate(hl.agg.collect_as_set(gnomad_indels_not_in_CADD.locus.contig))


#gnomad_indels_not_in_CADD_mitoStripped.aggregate(
    #hl.agg.collect_as_set(gnomad_indels_not_in_CADD_mitoStripped.locus.contig))
#gnomad_indels_not_in_CADD_mitoStripped.count()


#export_indels_for_CADD_analysis(gnomad_indels_not_in_CADD_mitoStripped, "gs://gnomad-wphu/CADD_indels_for_upload_mitoStripped.vcf.bgz")

In [29]:
#test = gnomad_indels_not_in_CADD.head(150000)
#export_indels_for_CADD_analysis(test, "gs://gnomad-wphu/CADD_indels_for_upload_test_90000.vcf")


In [30]:
#CADD_snps_test = load_CADD(path = "gs://gnomad-wphu/whole_genome_SNVs.tsv", n_partitions=5000)
#CADD_snps_test.show()

#CADD_snps = CADD_snps.checkpoint("gs://gnomad-wphu/CADD-v1.6-SNVs.ht")

In [31]:
#cadd_ht = hl.import_table("gs://gnomad-wphu/whole_genome_SNVs.tsv", comment="#", no_header=True, min_partitions=5000)

# Primate AI

In [32]:
#primate_ai_info.count()

In [34]:
#primate_ai_info_new = create_primate_ai_info()

In [35]:
#primate_ai_info_new = primate_ai_info_new.repartition(500)
#primate_ai_info_new = primate_ai_info_new.checkpoint("gs://gnomad-wphu/primate-ai-info.ht", overwrite=False)

# Combined Annotations

In [36]:
#gnomad_variants.count()
#gnomad_variants.describe()
#gnomad_variants.show()

In [37]:
#seqr_annotations_tbl.show()
#seqr_annotations_tbl.describe()

In [11]:
#result = result.checkpoint("gs://gnomad-wphu/gnomad-3.1-all-variants-annotations.ht")
result.write("gs://gnomad/annotations/hail-0.2/ht/genomes_v3.1/gnomad_genomes_v3.1.analyst_annotations.ht", overwrite=True)


2020-10-28 18:07:02 Hail: INFO: wrote table with 1584091437 rows in 7505 partitions to gs://gnomad/annotations/hail-0.2/ht/genomes_v3.1/gnomad_genomes_v3.1.analyst_annotations.ht


In [73]:
result.show()

+---------------+------------+----------------+------------+-------------------+--------------+--------------+---------------------+
| locus         | alleles    | cadd.raw_score | cadd.phred | revel.revel_score | revel.ref_aa | revel.alt_aa | splice_ai.splice_ai |
+---------------+------------+----------------+------------+-------------------+--------------+--------------+---------------------+
| locus<GRCh38> | array<str> |        float32 |    float32 |           float64 | str          | str          | array<float32>      |
+---------------+------------+----------------+------------+-------------------+--------------+--------------+---------------------+
| chr1:10001    | ["T","A"]  |       7.03e-01 |   8.48e+00 |                NA | NA           | NA           | NA                  |
| chr1:10001    | ["T","C"]  |       7.51e-01 |   8.92e+00 |                NA | NA           | NA           | NA                  |
| chr1:10007    | ["T","C"]  |       7.56e-01 |   8.97e+00 |                NA | NA           | NA           | NA                  |
| chr1:10009    | ["A","C"]  |       7.20e-01 |   8.64e+00 |                NA | NA           | NA           | NA                  |
| chr1:10013    | ["T","A"]  |       7.02e-01 |   8.47e+00 |                NA | NA           | NA           | NA                  |
| chr1:10015    | ["A","C"]  |       7.14e-01 |   8.58e+00 |                NA | NA           | NA           | NA                  |
| chr1:10019    | ["T","C"]  |       7.51e-01 |   8.92e+00 |                NA | NA           | NA           | NA                  |
| chr1:10020    | ["A","C"]  |       7.13e-01 |   8.57e+00 |                NA | NA           | NA           | NA                  |
| chr1:10021    | ["A","C"]  |       7.13e-01 |   8.58e+00 |                NA | NA           | NA           | NA                  |
| chr1:10022    | ["C","T"]  |       6.21e-01 |   7.71e+00 |                NA | NA           | NA           | NA                  |
+---------------+------------+----------------+------------+-------------------+--------------+--------------+---------------------+

+------------------+------------------------------+-----------------------------+
| splice_ai.max_ds | splice_ai.splice_consequence | primate_ai.primate_ai_score |
+------------------+------------------------------+-----------------------------+
|          float32 | str                          |                     float32 |
+------------------+------------------------------+-----------------------------+
|               NA | NA                           |                          NA |
|               NA | NA                           |                          NA |
|               NA | NA                           |                          NA |
|               NA | NA                           |                          NA |
|               NA | NA                           |                          NA |
|               NA | NA                           |                          NA |
|               NA | NA                           |                          NA |
|               NA | NA                           |                          NA |
|               NA | NA                           |                          NA |
|               NA | NA                           |                          NA |
+------------------+------------------------------+-----------------------------+
showing top 10 rows

# Other Things

In [ ]:
#missing_cadd_scores = result.filter(hl.is_defined(result.cadd.phred),keep=False)
#missing_cadd_scores.count()

In [ ]:
#missing_cadd_scores.filter(missing_cadd_scores.locus.contig=="chrM", keep=False).count()

In [ ]:
#primate_ai_info.ExAC_coverage.summarize()

In [ ]:
#print(gnomad_indels_in_CADD.count())
#print(gnomad_indels_not_in_CADD.count())
#print(gnomad_indels.count())

#misisng_cadd_scores = missing_cadd_scores.write("gs://gnomad-tmp/missing_cadd_scores.ht", overwrite=True)
#missing_cadd_scores = hl.read_table("gs://gnomad-tmp/missing_cadd_scores.ht")

In [ ]:
#missing_cadd_scores_no_M = missing_cadd_scores.filter(missing_cadd_scores.locus.contig=="chrM",keep=False)
#print(missing_cadd_scores.count())
#print(missing_cadd_scores_no_M.count())

#print(missing_cadd_scores.filter(missing_cadd_scores.locus.contig=="chrM",keep=True).count())
#print(gnomad_variants.filter(gnomad_variants.locus.contig=="chrM", keep=True).count())
#missing_cadd_scores_no_M.alleles.collect()

In [ ]:
#hl.len(missing_cadd_scores_no_M.old_alleles).summarize()
#missing_cadd_scores_no_M.count()
#hl.len(missing_cadd_scores_no_M.old_alleles).show()
#missing_cadd_scores_no_M.select(missing_cadd_scores_no_M.old_alleles).write("gs://gnomad-tmp/gnomad-31-complex-variants.ht")
#missing_cadd_scores_no_M.select(missing_cadd_scores_no_M.old_alleles).export("gs://gnomad-tmp/gnomad-31-complex-variants.tsv")

In [ ]:
#missing_cadd_scores_diff_key = missing_cadd_scores_no_M.select(missing_cadd_scores_no_M.old_alleles)
#missing_cadd_scores_diff_key = missing_cadd_scores_diff_key.key_by(missing_cadd_scores_diff_key.old_alleles)
#missing_cadd_scores_diff_key = missing_cadd_scores_diff_key.collect_by_key()

In [ ]:
#hl.len(missing_cadd_scores_diff_key.values)>1

In [ ]:
#len(set([frozenset(x) for x in missing_cadd_scores_no_M.old_alleles.collect()]))

In [ ]:
#missing_cadd_scores_diff_key.export("gs://gnomad-tmp/gnomad-variants-complex-by-old-alleles.tsv")

In [ ]:
#missing_cadd_scores_diff_key.count()

In [ ]:
#export_for_CADD_analysis(missing_cadd_scores_no_M, "gs://gnomad-wphu/gnomad-large-deletions-cadd.vcf")

In [ ]:
#example = ['AGGCTGACCTCTGTCCGCGTGGGAGGGGCCGGTGTGAGGCAAGGGGCTCAGGCTGACCTCTGTCCGCGTGGGAGGGGCCGGTGTGAGGCAAGGGGCTCAGGCTGACCTCTGTCCGCGTGGGAGGGGCCGGGGTGAGGCAAGGGCTCACACTGACCTCTCTCAGCGTGGGAGGGGCCGGTGTGAGGCAAGGGGCTCGGGCTGACCTCTCTCAGCGTGGGAGGGGCCGGTGTGAGGCAAGGGGCTCGGGCTGACCTCTCTCAGCGTGGGAGGGGCCGGTGTGAGGCAAGGGGCTCG', 'G']
#(~(hl.is_snp(example[0], example[1]))).show()
#hl.is_indel(example[0], example[1]).show()

In [ ]:
#not_snps = gnomad_variants.filter(hl.is_snp(gnomad_variants.alleles[0],gnomad_variants.alleles[1]),keep=False)
#not_snps.count()

In [ ]:
#gnomad_indels.count()

In [ ]:
#questionable = not_snps.anti_join(gnomad_indels)
#questionable.count()

In [ ]:
#print(missing_cadd_scores_no_M.semi_join(questionable).count())
#print(missing_cadd_scores_no_M.filter(hl.is_complex(missing_cadd_scores_no_M.alleles[0], missing_cadd_scores_no_M.alleles[1])).count())


In [ ]:
#hl.filter_intervals(questionable,spliceAi_genes.INTERVAL.collect()).select().export("gs://gnomad-wphu/missing-splice-ai-variants.tsv")


In [ ]:
#gnomad_variants.join(CADD_snps.transmute(CADD = hl.struct(RawScore = CADD_snps.RawScore, PHRED = CADD_snps.PHRED)), how="left").show()
#CADD_snps.transmute(CADD = hl.struct(RawScore = CADD_snps.RawScore, PHRED = CADD_snps.PHRED))


In [ ]:
#spliceAi_info_skip_invalid.describe()
#spliceAi_info_skip_invalid.rsid.show()
#spliceAi_info_skip_invalid.qual.show()
#spliceAi_info_skip_invalid.filters.show()
#spliceAi_info_skip_invalid.info.show()

#reannotate = spliceAi_info_skip_invalid.annotate_rows(info = spliceAi_info_skip_invalid.info.annotate(
#                                                risd = spliceAi_info_skip_invalid.rsid,
#                                                qual = spliceAi_info_skip_invalid.qual,
#                                                filters = spliceAi_info_skip_invalid.filters
#                                                )
#                                        )

#reannotate = reannotate.rename({"info" : "SPLICE_AI"})
#reannotate.describe()

In [ ]:
#gnomadgnomad3_release.filter(hl.is_indel(gnomad3_release.alleles[0], gnomad3_release.alleles[1]))

In [ ]:
#gnomad_31_CADD_indels = hl.import_table("gs://gnomad-julia/gnomad_v3.1/cadd_indel_output/CADD_gnomad3.1_scores_*.tsv.gz", comment="#",no_header=True, force_bgz=True)
#gnomad_31_CADD_indels = load_CADD("gs://gnomad-julia/gnomad_v3.1/cadd_indel_output/CADD_gnomad3.1_scores_*.tsv.gz", n_partitions=3000, force_bgz=True)



In [ ]:
#gnomad_31_CADD_complex = load_CADD("gs://gnomad-julia/gnomad_v3.1/cadd_indel_output_extra/CADD_gnomad3.1_scores_*.tsv.gz",n_partitions=3000, force_bgz=True)
#gnomad_31_CADD_complex.describe()
#gnomad_31_CADD_complex.checkpoint("gs://gnomad-wphu/CADD-1.6-gnomad-complex-variants.ht")


In [ ]:
#gnomad_31_CADD_complex.count()

In [ ]:
#gnomad_31_CADD_indels.count()

In [ ]:
#gnomad_indels_not_in_CADD.filter(gnomad_indels_not_in_CADD.locus.contig=="chrM",keep=False).count()

In [ ]:
#gnomad_indels_not_in_CADD.count()

In [ ]:
#gnomad_indels_not_in_CADD.filter(gnomad_indels_not_in_CADD.locus.contig=="chrM",keep=False).anti_join(gnomad_31_CADD_indels).count()

In [ ]:
#gnomad_31_CADD_indels = gnomad_31_CADD_indels.checkpoint("gs://gnomad-wphu/CADD-indels-gnomad.3.1.ht")

In [ ]:
#unified = make_unified_CADD()

In [ ]:
#unified.describe()
#unified.show()

In [ ]:
#unified.filePathSource["snvs"].show()

In [ ]:
#unified = unified.checkpoint("gs://gnomad-wphu/complete-CADD-v1.6-annotations.ht")

In [ ]:
#unified.count()

In [ ]:
#import requests
#r = requests.get(url = "https://spliceailookup-api.broadinstitute.org/spliceai/?hg=38&variant=chr8-140300616-T-G")
#print(r.json())

In [ ]:
#test = missing_cadd_scores_no_M.select().head(5).collect()

In [ ]:
#print(test)

In [ ]:
#test[0]

In [ ]:
#primate_ai_info.show()
#primate_ai_info_old.show()

In [ ]:
#spliceAi_info_skip_invalid.describe()

In [ ]:
#test = hl.import_vcf("gs://seqr-reference-data/GRCh38/primate_ai/PrimateAI_scores_v0.2.liftover_grch38.vcf.gz", force_bgz=True)

In [ ]:
#test.describe()

In [ ]:
#test2 = hl.read('gs://seqr-reference-data/GRCh38/primate_ai/PrimateAI_scores_v0.2.liftover_grch38.vds')

In [ ]:
#spliceAi_info_skip_invalid.qual.summarize()

In [7]:
dbNSFP = hl.import_table("gs://gnomad-wphu/dbNSFP/dbNSFP4.1/dbNSFP4.1a_variant.chr*.gz", force=True, missing='.')

2020-10-22 03:14:17 Hail: WARN: file 'gs://gnomad-wphu/dbNSFP/dbNSFP4.1/dbNSFP4.1a_variant.chr1.gz' is 2.8G
  It will be loaded serially (on one core) due to usage of the 'force' argument.
  If it is actually block-gzipped, either rename to .bgz or use the 'force_bgz'
  argument.
2020-10-22 03:14:18 Hail: WARN: file 'gs://gnomad-wphu/dbNSFP/dbNSFP4.1/dbNSFP4.1a_variant.chr10.gz' is 1.1G
  It will be loaded serially (on one core) due to usage of the 'force' argument.
  If it is actually block-gzipped, either rename to .bgz or use the 'force_bgz'
  argument.
2020-10-22 03:14:18 Hail: WARN: file 'gs://gnomad-wphu/dbNSFP/dbNSFP4.1/dbNSFP4.1a_variant.chr11.gz' is 1.7G
  It will be loaded serially (on one core) due to usage of the 'force' argument.
  If it is actually block-gzipped, either rename to .bgz or use the 'force_bgz'
  argument.
2020-10-22 03:14:18 Hail: WARN: file 'gs://gnomad-wphu/dbNSFP/dbNSFP4.1/dbNSFP4.1a_variant.chr12.gz' is 1.5G
  It will be loaded serially (on one core) due

In [22]:
dbNSFP.REVEL_score.summarize()

Non-missing,73967181 (88.04%)
Missing,10045912 (11.96%)
Min Size,5
Max Size,5
Mean Size,5.00
Sample Values,"['0.206', '0.253', '0.206', '0.046', '0.022']"


In [12]:
revel_information_grch38.show()

,,,,
locus,alleles,REVEL,aaref,aaalt
locus<GRCh38>,array<str>,float64,str,str
chr1:35142,"[""G"",""A""]",2.70e-02,"""T""","""M"""
chr1:35142,"[""G"",""C""]",3.50e-02,"""T""","""R"""
chr1:35142,"[""G"",""T""]",4.30e-02,"""T""","""K"""
chr1:35143,"[""T"",""A""]",1.80e-02,"""T""","""S"""
chr1:35143,"[""T"",""C""]",3.40e-02,"""T""","""A"""
chr1:35143,"[""T"",""G""]",3.90e-02,"""T""","""P"""
chr1:35144,"[""A"",""C""]",1.20e-02,"""C""","""W"""
chr1:35145,"[""C"",""A""]",2.30e-02,"""C""","""F"""


In [18]:
revel_information_grch38.count()

82077491

In [19]:
gnomad_variants.count()

1570602305

In [20]:
revel_information_grch38.anti_join(gnomad_variants).count()

72983629

In [8]:
dbNSFP = dbNSFP.repartition(500)
#dbNSFP.n_partitions()
dbNSFP = dbNSFP.checkpoint("gs://gnomad-wphu/dbNSFP.ht")

2020-10-22 04:13:41 Hail: INFO: wrote table with 84013093 rows in 500 partitions to gs://gnomad-wphu/dbNSFP.ht


In [25]:
82077491-84013093

-1935602

In [28]:
revel_information = hl.import_table("gs://gnomad-wphu/revel_grch38_all_chromosomes.csv", delimiter=",", types={'hg19_pos':hl.tint,'grch38_pos':hl.tstr,'REVEL': hl.tfloat64})
revel_information.count()

2020-10-22 02:58:51 Hail: INFO: Reading table without type imputation
  Loading field 'chr' as type str (not specified)
  Loading field 'hg19_pos' as type int32 (user-supplied)
  Loading field 'grch38_pos' as type str (user-supplied)
  Loading field 'ref' as type str (not specified)
  Loading field 'alt' as type str (not specified)
  Loading field 'aaref' as type str (not specified)
  Loading field 'aaalt' as type str (not specified)
  Loading field 'REVEL' as type float64 (user-supplied)


82100677

In [29]:
82100677-82077491

23186

In [12]:
dbNSFP.VEST4_score.summarize()

Non-missing,81566269 (97.09%)
Missing,2446824 (2.91%)
Min Size,1
Max Size,209
Mean Size,13.96
Sample Values,"['0.213;.;.;0.271;.;0.268;.;.;.;.', '0.193;.;0.174;0.176;.;.;.', '0.772;0.779;0.781', '0.295;0.477;0.545', '0.017;0.017;0.024']"


In [9]:
full_annotations.revel.summarize()

Non-missing,9742241 (0.62%)
Missing,1574349196 (99.38%)
Non-missing,9742241 (0.62%)
Missing,1574349196 (99.38%)
Minimum,0.00
Maximum,1.00
Mean,0.23
Std Dev,0.23
Non-missing,9742241 (0.62%)
Missing,1574349196 (99.38%)
Min Size,1


In [13]:
revel_information_grch38.anti_join(full_annotations).show()

,,,,
locus,alleles,REVEL,aaref,aaalt
locus<GRCh38>,array<str>,float64,str,str
chr1:35142,"[""G"",""A""]",2.70e-02,"""T""","""M"""
chr1:35142,"[""G"",""C""]",3.50e-02,"""T""","""R"""
chr1:35142,"[""G"",""T""]",4.30e-02,"""T""","""K"""
chr1:35143,"[""T"",""A""]",1.80e-02,"""T""","""S"""
chr1:35143,"[""T"",""C""]",3.40e-02,"""T""","""A"""
chr1:35143,"[""T"",""G""]",3.90e-02,"""T""","""P"""
chr1:35144,"[""A"",""C""]",1.20e-02,"""C""","""W"""
chr1:35145,"[""C"",""A""]",2.30e-02,"""C""","""F"""


In [14]:
full_annotations.filter(hl.is_snp(full_annotations.alleles[0], full_annotations.alleles[1])).count()

1317419242

In [15]:
revel_information_grch38.anti_join(full_annotations).count()

72983629

In [18]:
revel_information_grch38.aggregate(hl.agg.group_by(revel_information_grch38.locus.contig, hl.agg.count()))

{'chr11': 4850779,
 'chr5': 3738178,
 'chr22': 1783478,
 'chr8': 2880984,
 'chr19': 5156147,
 'chrY': 31551,
 'chr1': 8460922,
 'chr15': 2682896,
 'chr12': 4361562,
 'chr18': 1223623,
 'chr20': 1984314,
 'chr2': 6108302,
 'chr13': 1448194,
 'chr7': 3845947,
 'chr14': 2672346,
 'chr3': 4849223,
 'chr17': 4507899,
 'chr4': 3388218,
 'chr6': 4151134,
 'chr9': 3369753,
 'chrX': 3100887,
 'chr10': 3289465,
 'chr21': 841178,
 'chr16': 3350511}

In [34]:
unique_loci = revel_information_grch38.key_by().key_by("locus").distinct()
a = unique_loci.aggregate(hl.agg.group_by(unique_loci.locus.contig, hl.agg.count()))

In [35]:
unique_loci_gnomad = gnomad_variants.key_by().key_by("locus").distinct()
b = unique_loci_gnomad.aggregate(hl.agg.group_by(unique_loci_gnomad.locus.contig, hl.agg.count()))

In [36]:
type(a)
type(b)

dict

In [7]:
full_annotations.summarize()

Non-missing,1584091437 (100.00%)
Missing,0
Contig Counts,"{'chr11': 69489149, 'chr5': 93769047, 'chr22': 22076679, 'chr8': 77100610, 'chr19': 34268343, 'chrY': 14637245, 'chr1': 119509995, 'chr15': 43776955, 'chr12': 68722532, 'chr18': 38674164, 'chr20': 31905096, 'chr2': 124209991, 'chr13': 50807608, 'chr7': 83637854, 'chr14': 46794023, 'chr3': 101328983, 'chrM': 5746, 'chr17': 43009606, 'chr4': 99543980, 'chr6': 86614746, 'chr9': 66585932, 'chrX': 130280891, 'chr10': 69628678, 'chr21': 21073146, 'chr16': 46640438}"
Non-missing,1584091437 (100.00%)
Missing,0
Min Size,2
Max Size,2
Mean Size,2.00
Non-missing,3168182874 (100.00%)
Missing,0
Min Size,1


In [8]:
spliceAi_info_skip_invalid.

Non-missing,0
Missing,0


Non-missing,3696314616 (100.00%)
Missing,0
Contig Counts,"{'chr11': 176389931, 'chr5': 212300234, 'chr22': 56180502, 'chr8': 165598617, 'chr19': 98755025, 'chrY': 7909364, 'chr1': 324696513, 'chr15': 122307725, 'chr12': 185933382, 'chr18': 87511547, 'chr20': 84077613, 'chr2': 307151672, 'chr13': 95599997, 'chr7': 218906720, 'chr14': 114442374, 'chr3': 262597980, 'chr17': 135095070, 'chr4': 202720275, 'chr6': 211688500, 'chr9': 150691302, 'chrX': 147792183, 'chr10': 182471028, 'chr21': 37698988, 'chr16': 107798074}"
Non-missing,3696314616 (100.00%)
Missing,0
Min Size,2
Max Size,2
Mean Size,2.00
Non-missing,7392629232 (100.00%)
Missing,0
Min Size,1


Entries
======
0 records.

In [10]:
full_annotations.filter(hl.is_snp(full_annotations.alleles[0],full_annotations.alleles[1])).splice_ai.summarize()

Non-missing,482841462 (36.65%)
Missing,834577780 (63.35%)
Non-missing,482841462 (36.65%)
Missing,834577780 (63.35%)
Min Size,4
Max Size,4
Mean Size,4.00
Non-missing,1931365848 (100.00%)
Missing,0
Minimum,0.00
Maximum,1.00


In [11]:
full_annotations.filter(~hl.is_snp(full_annotations.alleles[0],full_annotations.alleles[1])).splice_ai.summarize()

Non-missing,262288960 (98.36%)
Missing,4383235 (1.64%)
Non-missing,100955534 (37.86%)
Missing,165716661 (62.14%)
Min Size,4
Max Size,4
Mean Size,4.00
Non-missing,403822136 (100.00%)
Missing,0
Minimum,-0.00
Maximum,1.00


In [12]:
full_annotations.filter(hl.is_indel(full_annotations.alleles[0],full_annotations.alleles[1])).splice_ai.summarize()

Non-missing,262288960 (98.36%)
Missing,4380928 (1.64%)
Non-missing,100955534 (37.86%)
Missing,165714354 (62.14%)
Min Size,4
Max Size,4
Mean Size,4.00
Non-missing,403822136 (100.00%)
Missing,0
Minimum,-0.00
Maximum,1.00


In [13]:
full_annotations.filter(hl.is_indel(full_annotations.alleles[0],full_annotations.alleles[1])).splice_ai.show()

+---------------+------------------------------------------+---------------------+------------------+------------------------------+
| locus         | alleles                                  | splice_ai.splice_ai | splice_ai.max_ds | splice_ai.splice_consequence |
+---------------+------------------------------------------+---------------------+------------------+------------------------------+
| locus<GRCh38> | array<str>                               | array<float32>      |          float32 | str                          |
+---------------+------------------------------------------+---------------------+------------------+------------------------------+
| chr1:10055    | ["T","TAACCCTAACCCTAACCCTAACCCTAACCCT... | NA                  |               NA | NA                           |
| chr1:10055    | ["T","TAACCCTAACCCTAACCCTAACCCTAACCCT... | NA                  |               NA | NA                           |
| chr1:10061    | ["T","TAACCCTAACCCTAACCCTAACCCTAACCCT... | NA                  |               NA | NA                           |
| chr1:10064    | ["C","CCCTAACCCTAACCCTAACCCTAACCCTAAC... | NA                  |               NA | NA                           |
| chr1:10067    | ["T","TAACCCTAACCCTAACCCTAACCCTAACCCT... | NA                  |               NA | NA                           |
| chr1:10067    | ["T","TAACCCTAACCCTAACCCTAACCCTAACCCT... | NA                  |               NA | NA                           |
| chr1:10073    | ["T","TAACCCTAACCCTAACCCTAACCCTAACCCT... | NA                  |               NA | NA                           |
| chr1:10079    | ["T","TAACCCTAACCCTAACCCTAACCCTAACCC"]   | NA                  |               NA | NA                           |
| chr1:10091    | ["T","TAACCCTAACCCTAACCC"]               | NA                  |               NA | NA                           |
| chr1:10108    | ["C","CA"]                               | NA                  |               NA | NA                           |
+---------------+------------------------------------------+---------------------+------------------+------------------------------+
showing top 10 rows

In [20]:
a = full_annotations.filter(hl.is_indel(full_annotations.alleles[0],full_annotations.alleles[1]))
b = a.filter(hl.is_missing(a.splice_ai))
b.show()
c = a.filter(hl.is_defined(a.splice_ai))
c.show()
b.splice_ai.summarize()
c.splice_ai.summarize()

+---------------+------------------------------------------+----------------+------------+-------------------+--------------+--------------+
| locus         | alleles                                  | cadd.raw_score | cadd.phred | revel.revel_score | revel.ref_aa | revel.alt_aa |
+---------------+------------------------------------------+----------------+------------+-------------------+--------------+--------------+
| locus<GRCh38> | array<str>                               |        float32 |    float32 |           float64 | str          | str          |
+---------------+------------------------------------------+----------------+------------+-------------------+--------------+--------------+
| chr1:179816   | ["G","GA"]                               |      -5.50e-02 |   1.03e+00 |                NA | NA           | NA           |
| chr1:179885   | ["A","AGCT"]                             |      -4.31e-02 |   1.09e+00 |                NA | NA           | NA           |
| chr1:180054   | ["TTTA","T"]                             |      -9.53e-02 |   8.48e-01 |                NA | NA           | NA           |
| chr1:180088   | ["TACCCTA","T"]                          |      -1.03e-01 |   8.16e-01 |                NA | NA           | NA           |
| chr1:180088   | ["TACCCTAACCCTAACCCTAACCCCTAATCCTAACC... |      -2.33e-01 |   4.33e-01 |                NA | NA           | NA           |
| chr1:180089   | ["ACCCTAACCCTAACCCTAACCCCTAATCCTAACCC... |      -2.83e-01 |   3.39e-01 |                NA | NA           | NA           |
| chr1:180094   | ["AACCCTAACCCTAACCCCTAATCCTAACCCTAATC... |      -2.73e-01 |   3.56e-01 |                NA | NA           | NA           |
| chr1:180095   | ["A","AC"]                               |      -3.15e-02 |   1.15e+00 |                NA | NA           | NA           |
| chr1:180095   | ["ACCCTAACCCTAACCCCTAATCCTAACCCTAATCC... |      -2.84e-01 |   3.36e-01 |                NA | NA           | NA           |
| chr1:180096   | ["CCCTAACCCTAACCCCTAAT","C"]             |      -1.58e-01 |   6.27e-01 |                NA | NA           | NA           |
+---------------+------------------------------------------+----------------+------------+-------------------+--------------+--------------+

+---------------------+------------------+------------------------------+-----------------------------+
| splice_ai.splice_ai | splice_ai.max_ds | splice_ai.splice_consequence | primate_ai.primate_ai_score |
+---------------------+------------------+------------------------------+-----------------------------+
| array<float32>      |          float32 | str                          |                     float32 |
+---------------------+------------------+------------------------------+-----------------------------+
| NA                  |               NA | NA                           |                          NA |
| NA                  |               NA | NA                           |                          NA |
| NA                  |               NA | NA                           |                          NA |
| NA                  |               NA | NA                           |                          NA |
| NA                  |               NA | NA                           |                          NA |
| NA                  |               NA | NA                           |                          NA |
| NA                  |               NA | NA                           |                          NA |
| NA                  |               NA | NA                           |                          NA |
| NA                  |               NA | NA                           |                          NA |
| NA                  |               NA | NA                           |                          NA |
+---------------------+------------------+------------------------------+-----------------------------+
showing top 10 rows

+---------------+------------------------------------------+----------------+------------+-------------------+--------------+--------------+
| locus         | alleles                                  | cadd.raw_score | cadd.phred | revel.revel_score | revel.ref_aa | revel.alt_aa |
+---------------+------------------------------------------+----------------+------------+-------------------+--------------+--------------+
| locus<GRCh38> | array<str>                               |        float32 |    float32 |           float64 | str          | str          |
+---------------+------------------------------------------+----------------+------------+-------------------+--------------+--------------+
| chr1:10055    | ["T","TAACCCTAACCCTAACCCTAACCCTAACCCT... |       1.92e-01 |   3.03e+00 |                NA | NA           | NA           |
| chr1:10055    | ["T","TAACCCTAACCCTAACCCTAACCCTAACCCT... |       1.92e-01 |   3.03e+00 |                NA | NA           | NA           |
| chr1:10061    | ["T","TAACCCTAACCCTAACCCTAACCCTAACCCT... |       2.00e-01 |   3.12e+00 |                NA | NA           | NA           |
| chr1:10064    | ["C","CCCTAACCCTAACCCTAACCCTAACCCTAAC... |       1.90e-01 |   3.02e+00 |                NA | NA           | NA           |
| chr1:10067    | ["T","TAACCCTAACCCTAACCCTAACCCTAACCCT... |       2.51e-01 |   3.71e+00 |                NA | NA           | NA           |
| chr1:10067    | ["T","TAACCCTAACCCTAACCCTAACCCTAACCCT... |       2.25e-01 |   3.42e+00 |                NA | NA           | NA           |
| chr1:10073    | ["T","TAACCCTAACCCTAACCCTAACCCTAACCCT... |       2.51e-01 |   3.71e+00 |                NA | NA           | NA           |
| chr1:10079    | ["T","TAACCCTAACCCTAACCCTAACCCTAACCC"]   |       2.77e-01 |   4.02e+00 |                NA | NA           | NA           |
| chr1:10091    | ["T","TAACCCTAACCCTAACCC"]               |       3.28e-01 |   4.61e+00 |                NA | NA           | NA           |
| chr1:10108    | ["C","CA"]                               |       3.91e-01 |   5.32e+00 |                NA | NA           | NA           |
+---------------+------------------------------------------+----------------+------------+-------------------+--------------+--------------+

+---------------------+------------------+------------------------------+-----------------------------+
| splice_ai.splice_ai | splice_ai.max_ds | splice_ai.splice_consequence | primate_ai.primate_ai_score |
+---------------------+------------------+------------------------------+-----------------------------+
| array<float32>      |          float32 | str                          |                     float32 |
+---------------------+------------------+------------------------------+-----------------------------+
| NA                  |               NA | NA                           |                          NA |
| NA                  |               NA | NA                           |                          NA |
| NA                  |               NA | NA                           |                          NA |
| NA                  |               NA | NA                           |                          NA |
| NA                  |               NA | NA                           |                          NA |
| NA                  |               NA | NA                           |                          NA |
| NA                  |               NA | NA                           |                          NA |
| NA                  |               NA | NA                           |                          NA |
| NA                  |               NA | NA                           |                          NA |
| NA                  |               NA | NA                           |                          NA |
+---------------------+------------------+------------------------------+-----------------------------+
showing top 10 rows

Non-missing,0
Missing,4380928 (100.00%)


Non-missing,262288960 (100.00%)
Missing,0
Non-missing,100955534 (38.49%)
Missing,161333426 (61.51%)
Min Size,4
Max Size,4
Mean Size,4.00
Non-missing,403822136 (100.00%)
Missing,0
Minimum,-0.00
Maximum,1.00


In [21]:
print(b.splice_ai.summarize())
print(c.splice_ai.summarize())

Non-missing,0
Missing,4380928 (100.00%)


None


Non-missing,262288960 (100.00%)
Missing,0
Non-missing,100955534 (38.49%)
Missing,161333426 (61.51%)
Min Size,4
Max Size,4
Mean Size,4.00
Non-missing,403822136 (100.00%)
Missing,0
Minimum,-0.00
Maximum,1.00


None


In [22]:
spliceAi_indels_skip_invalid.summarize()

Non-missing,0
Missing,0


Non-missing,263071868 (100.00%)
Missing,0
Contig Counts,"{'chr11': 11664173, 'chr5': 15405197, 'chr22': 3618580, 'chr8': 12618223, 'chr19': 6793978, 'chrY': 1624633, 'chr1': 20097560, 'chr15': 7537321, 'chr12': 12114513, 'chr18': 6583523, 'chr20': 5545355, 'chr2': 20940975, 'chr13': 8585290, 'chr7': 14492115, 'chr14': 8163417, 'chr3': 17000241, 'chr17': 7893956, 'chr4': 16917567, 'chr6': 14979613, 'chr9': 10722816, 'chrX': 16407215, 'chr10': 12017810, 'chr21': 3498821, 'chr16': 7848976}"
Non-missing,263071868 (100.00%)
Missing,0
Min Size,2
Max Size,2
Mean Size,2.00
Non-missing,526143736 (100.00%)
Missing,0
Min Size,1


Entries
======
0 records.

In [9]:
full_annotations.summarize()

Non-missing,1584091437 (100.00%)
Missing,0
Contig Counts,"{'chr11': 69489149, 'chr5': 93769047, 'chr22': 22076679, 'chr8': 77100610, 'chr19': 34268343, 'chrY': 14637245, 'chr1': 119509995, 'chr15': 43776955, 'chr12': 68722532, 'chr18': 38674164, 'chr20': 31905096, 'chr2': 124209991, 'chr13': 50807608, 'chr7': 83637854, 'chr14': 46794023, 'chr3': 101328983, 'chrM': 5746, 'chr17': 43009606, 'chr4': 99543980, 'chr6': 86614746, 'chr9': 66585932, 'chrX': 130280891, 'chr10': 69628678, 'chr21': 21073146, 'chr16': 46640438}"
Non-missing,1584091437 (100.00%)
Missing,0
Min Size,2
Max Size,2
Mean Size,2.00
Non-missing,3168182874 (100.00%)
Missing,0
Min Size,1


In [8]:
primate_ai_test= hl.import_table("gs://gnomad-wphu/PrimateAI_scores_v0.2_hg38.tsv.gz",
                                          force_bgz=True, comment="#", skip_blank_lines=True,
                                          types={"pos":hl.tint32, 'primateDL_score':hl.tfloat32, 'ExAC_coverage':hl.tfloat32}
                                         )
primate_ai_test.show()

FatalError: ZipException: File does not conform to block gzip format.

Java stack trace:
java.util.zip.ZipException: File does not conform to block gzip format.
	at is.hail.io.compress.BGzipInputStream$BGzipHeader.<init>(BGzipInputStream.java:41)
	at is.hail.io.compress.BGzipInputStream.resetState(BGzipInputStream.java:256)
	at is.hail.io.compress.BGzipInputStream.<init>(BGzipInputStream.java:110)
	at is.hail.io.compress.BGzipInputStream.<init>(BGzipInputStream.java:122)
	at is.hail.io.fs.BGZipCompressionCodec$.makeInputStream(FS.scala:57)
	at is.hail.io.fs.FS$class.open(FS.scala:141)
	at is.hail.io.fs.HadoopFS.open(HadoopFS.scala:70)
	at is.hail.io.fs.FS$class.open(FS.scala:148)
	at is.hail.io.fs.HadoopFS.open(HadoopFS.scala:70)
	at is.hail.io.fs.FS$class.readLines(FS.scala:208)
	at is.hail.io.fs.HadoopFS.readLines(HadoopFS.scala:70)
	at is.hail.expr.ir.TextTableReader$.readMetadata(TextTableReader.scala:268)
	at is.hail.expr.ir.TextTableReader$.apply(TextTableReader.scala:306)
	at is